# Descargar los vídeos de YouTube

In [5]:
import os
import yt_dlp
import time

# -------- CONFIGURACIÓN --------
video_urls = [
    "https://www.youtube.com/watch?v=rNwnEwlsNSA",
    "https://www.youtube.com/watch?v=5MgBikgcWnY",
    # Añade aquí más URLs de videos
]

raw_dir = "data/raw"
processed_dir = "data/processed"
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)


# -------- DESCARGAR VIDEOS --------
for video_url in video_urls:
    try:
        # Extraer el ID del video para el nombre del archivo
        with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
            info = ydl.extract_info(video_url, download=False)
            video_id = info['id']
            video_ext = info['ext']
            
        nombre_base = f"tedtalk_{video_id}"
        input_path = os.path.join(raw_dir, f"{nombre_base}.{video_ext}")
        output_path = os.path.join(processed_dir, f"landmarks_{nombre_base}.mp4")

        if not os.path.exists(input_path):
            print(f"\n🔽 Descargando: {video_url}")
            ydl_opts = {
                'outtmpl': input_path,
                'format': 'mp4',
                'quiet': True,
            }
            
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([video_url])
            
            # Verificar descarga exitosa
            time.sleep(1)
            if os.path.exists(input_path) and os.path.getsize(input_path) > 1024:
                print(f"✅ Descarga exitosa: {os.path.basename(input_path)} "
                      f"({os.path.getsize(input_path)/(1024*1024):.2f} MB)")
            else:
                print(f"❌ Error en descarga: {video_url}")
        else:
            print(f"📁 Archivo existente: {os.path.basename(input_path)}")

    except Exception as e:
        print(f"\n❌ Error procesando {video_url}: {str(e)}")
        continue

print("\n🔥 Proceso de descarga completado")


🔽 Descargando: https://www.youtube.com/watch?v=rNwnEwlsNSA
✅ Descarga exitosa: tedtalk_rNwnEwlsNSA.webm (64.17 MB)    

🔽 Descargando: https://www.youtube.com/watch?v=5MgBikgcWnY
✅ Descarga exitosa: tedtalk_5MgBikgcWnY.webm (58.38 MB)    

🔥 Proceso de descarga completado


# Procesamiento con MediaPipe Holistic

In [ ]:
import os
import cv2
import mediapipe as mp
import pandas as pd

# --- Rutas ---
raw_dir = "data/raw"
processed_dir = "data/processed"
landmarks_dir = os.path.join(processed_dir, "landmarks_csv")
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(landmarks_dir, exist_ok=True)

# --- Inicializar MediaPipe Holistic ---
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

holistic = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    enable_segmentation=False,
    refine_face_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# --- Obtener lista de vídeos .mp4 ---
video_files = [f for f in os.listdir(raw_dir) if f.endswith(".webm")]
if not video_files:
    print("⚠️ No se encontraron videos en data/raw/")
else:
    for idx, filename in enumerate(video_files):
        video_id = f"{idx:03d}"
        input_path = os.path.join(raw_dir, filename)
        output_video = os.path.join(processed_dir, f"processed_{video_id}.mp4")
        output_csv = os.path.join(landmarks_dir, f"landmarks_{video_id}.csv")

        if os.path.exists(output_video) and os.path.exists(output_csv):
            print(f"📁 Ya procesado, saltando: {filename}")
            continue

        print(f"\n🎥 Procesando: {filename}")
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"❌ No se pudo abrir: {input_path}")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

        frame_count = 0
        landmark_data = []

        FRAME_SKIP = 2

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            if frame_count % FRAME_SKIP == 0:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_rgb.flags.writeable = False
                results = holistic.process(frame_rgb)
                frame.flags.writeable = True

                # Dibujo
                if results.face_landmarks:
                    mp_drawing.draw_landmarks(
                        frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=1, circle_radius=1),
                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

                if results.left_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                        connection_drawing_spec=mp_drawing_styles.get_default_hand_connections_style())

                if results.right_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                        connection_drawing_spec=mp_drawing_styles.get_default_hand_connections_style())

                if results.pose_landmarks:
                    mp_drawing.draw_landmarks(
                        frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=2),
                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=2))

                out.write(frame)
            
            frame_count += 1

            # Guardar landmarks como filas CSV
            row = {"frame": frame_count}
            for name, landmark_list in [("face", results.face_landmarks),
                                        ("left_hand", results.left_hand_landmarks),
                                        ("right_hand", results.right_hand_landmarks),
                                        ("pose", results.pose_landmarks)]:
                if landmark_list:
                    for i, lm in enumerate(landmark_list.landmark):
                        row[f"{name}_{i}_x"] = lm.x
                        row[f"{name}_{i}_y"] = lm.y
                        row[f"{name}_{i}_z"] = lm.z
                        row[f"{name}_{i}_vis"] = lm.visibility if name == "pose" else None
            landmark_data.append(row)

            frame_count += 1
            print(f"\r   Frame {frame_count}/{total_frames}", end="")

        cap.release()
        out.release()
        print(f"\n✅ Guardado: {output_video}")

        # Guardar CSV
        df = pd.DataFrame(landmark_data)
        df.to_csv(output_csv, index=False)
        print(f"📄 Landmarks guardados en: {output_csv}")

    holistic.close()
    cv2.destroyAllWindows()
    print("\n🎉 Todos los vídeos han sido procesados.")

I0000 00:00:1748284345.555286 2517962 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Max
W0000 00:00:1748284345.648887 2552924 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748284345.659763 2552924 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748284345.660961 2552924 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748284345.661011 2552926 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748284345.661197 2552929 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling supp


🎥 Procesando: tedtalk_5MgBikgcWnY.webm
   Frame 34963/34963
✅ Guardado: data/processed/processed_000.mp4
📄 Landmarks guardados en: data/processed/landmarks_csv/landmarks_000.csv

🎥 Procesando: tedtalk_rNwnEwlsNSA.webm
   Frame 34814/34814
✅ Guardado: data/processed/processed_001.mp4
📄 Landmarks guardados en: data/processed/landmarks_csv/landmarks_001.csv

🎉 Todos los vídeos han sido procesados.


# Preprocesado de los .csv

In [9]:
import os
import pandas as pd
from IPython.display import display

# Directorio donde están los CSV de landmarks
landmarks_dir = "data/processed/landmarks_csv"
preprocessed_dir = "data/processed/preprocessed_csv"
os.makedirs(preprocessed_dir, exist_ok=True)

# Obtener lista de archivos CSV
csv_files = [f for f in os.listdir(landmarks_dir) if f.endswith(".csv")]
summary = []

for filename in csv_files:
    file_path = os.path.join(landmarks_dir, filename)
    df = pd.read_csv(file_path)

    # Rellenar NaN con interpolación (por columna)
    df_interp = df.interpolate(method='linear', limit_direction='both', axis=0)

    # Eliminar columnas con todos los valores nulos (por si hay landmarks no detectados nunca)
    df_clean = df_interp.dropna(axis=1, how='all')

    # Guardar CSV preprocesado
    output_filename = f"preprocessed_{filename}"
    output_path = os.path.join(preprocessed_dir, output_filename)
    df_clean.to_csv(output_path, index=False)

    summary.append({"archivo": filename, "frames": len(df), "columnas_finales": df_clean.shape[1]})

display(summary)


[{'archivo': 'landmarks_000.csv', 'frames': 34963, 'columnas_finales': 1693},
 {'archivo': 'landmarks_001.csv', 'frames': 34814, 'columnas_finales': 1693}]

# Análisis de ciertas métricas de MediaPipe

In [10]:
import os
import pandas as pd
import numpy as np
import math

# --- Ruta de entrada y salida ---
preprocessed_dir = "data/processed/preprocessed_csv"
metrics_dir = "data/processed/metrics"
os.makedirs(metrics_dir, exist_ok=True)

# --- Funciones auxiliares ---
def calculate_mar(row):
    try:
        # MAR = distancia vertical / horizontal entre labios (usamos puntos aproximados)
        top = np.array([row["face_13_x"], row["face_13_y"]])
        bottom = np.array([row["face_14_x"], row["face_14_y"]])
        left = np.array([row["face_61_x"], row["face_61_y"]])
        right = np.array([row["face_291_x"], row["face_291_y"]])
        vertical = np.linalg.norm(top - bottom)
        horizontal = np.linalg.norm(left - right)
        return vertical / horizontal if horizontal > 1e-6 else np.nan
    except:
        return np.nan

def average_wrist_y(row):
    ys = []
    for hand in ["left_hand", "right_hand"]:
        key = f"{hand}_0_y"  # Landmark 0 = wrist
        if key in row and not pd.isna(row[key]):
            ys.append(row[key])
    return np.mean(ys) if ys else np.nan

def shoulder_distance(row):
    try:
        left = np.array([row["pose_11_x"], row["pose_11_y"]])  # Left shoulder
        right = np.array([row["pose_12_x"], row["pose_12_y"]]) # Right shoulder
        return np.linalg.norm(left - right)
    except:
        return np.nan

def movement_energy(df, cols):
    diffs = df[cols].diff().fillna(0)
    energy = np.sqrt((diffs**2).sum(axis=1))
    return energy.rolling(window=5, min_periods=1).mean()

# --- Procesar CSVs ---
summary = []

for filename in os.listdir(preprocessed_dir):
    if not filename.endswith(".csv"):
        continue

    file_path = os.path.join(preprocessed_dir, filename)
    df = pd.read_csv(file_path)

    # MAR
    df["mar"] = df.apply(calculate_mar, axis=1)

    # Altura media muñecas (indicador de gesticulación)
    df["avg_wrist_y"] = df.apply(average_wrist_y, axis=1)

    # Distancia entre hombros (postura abierta o cerrada)
    df["shoulder_dist"] = df.apply(shoulder_distance, axis=1)

    # Energía de movimiento (sobre pose y manos)
    movement_cols = [col for col in df.columns if any(k in col for k in ["pose_", "left_hand_", "right_hand_"]) and col.endswith("_x") or col.endswith("_y")]
    df["movement_energy"] = movement_energy(df, movement_cols)

    # Guardar métricas como CSV
    output_path = os.path.join(metrics_dir, f"metrics_{filename}")
    df[["frame", "mar", "avg_wrist_y", "shoulder_dist", "movement_energy"]].to_csv(output_path, index=False)

    summary.append({
        "archivo": filename,
        "frames": len(df),
        "mar_mean": df["mar"].mean(),
        "wrist_y_mean": df["avg_wrist_y"].mean(),
        "shoulder_dist_mean": df["shoulder_dist"].mean(),
        "movement_energy_mean": df["movement_energy"].mean()
    })

# Mostrar resumen de métricas
summary_df = pd.DataFrame(summary)
display(summary_df)

,archivo,frames,mar_mean,wrist_y_mean,shoulder_dist_mean,movement_energy_mean
0,preprocessed_landmarks_001.csv,34814,0.092059,0.654256,0.140813,0.111531
1,preprocessed_landmarks_000.csv,34963,0.182070,0.679551,0.113705,0.119546


# Procesamiento con DeepFace

In [ ]:
import os
import cv2
import pandas as pd
from deepface import DeepFace
from tqdm import tqdm

# --- Carpetas ---
raw_dir = "data/raw"
emotion_dir = "data/processed/emotion_csv"
os.makedirs(emotion_dir, exist_ok=True)

# --- Parámetros ---
FRAME_INTERVAL = 1  # segundos entre frames a analizar

# --- Detectar vídeos ---
video_files = [f for f in os.listdir(raw_dir) if f.endswith(".webm")]
if not video_files:
    print("⚠️ No se encontraron vídeos en data/raw/")
else:
    for idx, filename in enumerate(video_files):
        input_path = os.path.join(raw_dir, filename)
        output_csv = os.path.join(emotion_dir, f"emotions_{idx:03d}.csv")

        if os.path.exists(output_csv):
            print(f"📁 Ya procesado: {filename}")
            continue

        print(f"\n🎥 Analizando emociones en: {filename}")
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"❌ No se pudo abrir: {input_path}")
            continue

        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps if fps > 0 else 0
        interval_frames = int(fps * FRAME_INTERVAL)

        data = []

        frame_id = 0
        pbar = tqdm(total=total_frames, desc="Procesando frames")

        while cap.isOpened():
            ret = cap.grab()  # avanzar al siguiente frame
            if not ret:
                break

            if frame_id % interval_frames == 0:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
                ret, frame = cap.read()
                if not ret:
                    break

                try:
                    analysis = DeepFace.analyze(frame, actions=['emotion', 'age', 'gender'], enforce_detection=False)
                    result = analysis[0] if isinstance(analysis, list) else analysis

                    dominant_emotion = result["dominant_emotion"]
                    emotion_score = result["emotion"][dominant_emotion]
                    age = result.get("age", None)
                    gender = result.get("gender", None)

                    data.append({
                        "frame_id": frame_id,
                        "timestamp_s": frame_id / fps,
                        "dominant_emotion": dominant_emotion,
                        "emotion_score": emotion_score,
                        "age": age,
                        "gender": gender
                    })

                except Exception as e:
                    print(f"⚠️ Error en frame {frame_id}: {e}")

            frame_id += 1
            pbar.update(1)

        cap.release()
        pbar.close()

        df = pd.DataFrame(data)
        df.to_csv(output_csv, index=False)
        print(f"✅ CSV guardado: {output_csv}")

print("\n🎉 Análisis emocional terminado.")

Procesando frames:   4%|▍         | 1392/34963 [00:40<07:18, 76.59it/s]